# Überschrift

In [ ]:
# Imports
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# CSV laden
df = pd.read_csv("../data/train.csv")  # relativer Pfad von notebooks/ aus

# Überblick
print("🔍 Überblick:")
print(df.shape)
print(df.info())
print(df.describe())

# Liste relevanter Features (Kategorien & numerisch diskret)
features = ["Pclass", "Sex", "SibSp", "Parch", "Embarked"]

for feature in features:
    print(f"\n📊 Überlebensrate nach {feature}:")
    print(df.groupby(feature)["Survived"].mean())
    sns.barplot(x=feature, y="Survived", data=df)
    plt.title(f"Überlebensrate nach {feature}")
    plt.show()

# Altersverteilung und Überleben (numerisch → mit Binning)
print("\n🧒 Altersverteilung (binned) und Überlebensrate:")
df["AgeBin"] = pd.cut(df["Age"], bins=[0, 12, 18, 30, 45, 60, 80], labels=["Kind", "Teen", "Jung", "Mittel", "Reif", "Alt"])
print(df.groupby("AgeBin")["Survived"].mean())
sns.barplot(x="AgeBin", y="Survived", data=df)
plt.title("Überlebensrate nach Altersgruppen")
plt.show()

# Fare binned
print("\n💰 Fare-Verteilung (binned) und Überlebensrate:")
df["FareBin"] = pd.qcut(df["Fare"], 4, labels=["Niedrig", "Mittel", "Hoch", "Sehr hoch"])
print(df.groupby("FareBin")["Survived"].mean())
sns.barplot(x="FareBin", y="Survived", data=df)
plt.title("Überlebensrate nach Ticketpreis")
plt.show()



🎯 Genauigkeit auf Testdaten: 0.797
